In [1]:
# All necessary libraries must be put here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg

In [2]:
# Read the CRSP Data and make it become a dataframe
crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")

# Check the data
crsp_data_1993_2019.head(5)

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_90371/4244800754.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")


,PERMNO,HdrCUSIP,Ticker,PERMCO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,10001,36720410,GFGC,7953,1993-01-04,150.0,14.5,14.50,14.5,14.50
1,10001,36720410,GFGC,7953,1993-01-05,0.0,NaN,NaN,NaN,NaN
2,10001,36720410,GFGC,7953,1993-01-06,0.0,NaN,NaN,NaN,NaN
3,10001,36720410,GFGC,7953,1993-01-07,228.0,14.5,14.50,14.5,14.50
4,10001,36720410,GFGC,7953,1993-01-08,1375.0,14.5,14.25,14.5,14.25


In [6]:
# At this part, we focus first on training sample
# Training data are taken from 1993 to 2000. 

crsp_data_1993_2019["DlyCalDt"] = pd.to_datetime(crsp_data_1993_2019["DlyCalDt"])

# Filter dates from 1993 to 2000
start_date = "1993-01-01"
end_date = "2000-12-31" 
crsp_data_1993_2000 = crsp_data_1993_2019[(crsp_data_1993_2019["DlyCalDt"] >= start_date) & (crsp_data_1993_2019["DlyCalDt"] <= end_date)]

# Check the last 5 values
print(crsp_data_1993_2000.tail(5))

# Check the shape of the crsp_data_1993_2000
print(crsp_data_1993_2000.shape) # (17223740, 10)

# Check the shape of the crsp_data_1993_2019
print(crsp_data_1993_2019.shape) # (51131155, 10)

          PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  DlyClose  \
50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   15.3750   
50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   15.0625   
50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   15.0625   
50923822   93316  98960110   ZING    7469 2000-03-16    700.0   15.3750   
50923823   93316  98960110    NaN    7469 2000-03-17      NaN       NaN   

           DlyLow  DlyHigh  DlyOpen  
50923819  15.1250   15.375  15.2500  
50923820  15.0625   15.375  15.3750  
50923821  15.0625   15.375  15.0625  
50923822  15.0625   15.375  15.0625  
50923823      NaN      NaN      NaN  
(17223740, 10)
(51131155, 10)


In [12]:
# Drop all the NaN

crsp_data_1993_2000 = crsp_data_1993_2000.dropna()

# Check the shape again
print(crsp_data_1993_2000.shape) # (15555109, 10)

(15555109, 10)


In [26]:
# Drop unused columns in the dataframe
crsp_data_1993_2000 = crsp_data_1993_2000.drop(["HdrCUSIP", "Ticker", "PERMCO"], axis = 1)

# Check the shape again
print(crsp_data_1993_2000.shape) # (15555109, 7)

(15555109, 7)


In [27]:
# Function to calculate 5-day log returns
def calculate_5_day_log_returns(group):
    # Create a column for the price 5 days later
    group["DlyClose_5_days_later"] = group["DlyClose"].shift(-5)

    # Calculate Log Returns
    group["Log_Return"] = np.log(group["DlyClose_5_days_later"] / group["DlyClose"])

    return group 

In [28]:
# Calculate 5-day stock returns for the whole dataset from 1993 to 2000
crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)

# Reset the index
crsp_data_1993_2000_R5 = crsp_data_1993_2000_R5.reset_index(drop = True)

# Display the result
crsp_data_1993_2000_R5

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_90371/4093158164.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)


,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,DlyClose_5_days_later,Log_Return
0,10001,1993-01-04,150.0,14.5000,14.5000,14.500,14.5000,13.75,-0.053110
1,10001,1993-01-07,228.0,14.5000,14.5000,14.500,14.5000,13.75,-0.053110
2,10001,1993-01-08,1375.0,14.5000,14.2500,14.500,14.2500,14.25,-0.017392
3,10001,1993-01-11,200.0,14.5000,14.5000,14.500,14.5000,14.25,-0.017392
4,10001,1993-01-13,700.0,13.7500,13.7500,14.000,14.0000,13.75,0.000000
...,...,...,...,...,...,...,...,...,...
15555104,93316,2000-03-10,8555.0,15.2500,15.1250,15.250,15.1250,NaN,NaN
15555105,93316,2000-03-13,10685.0,15.3750,15.1250,15.375,15.2500,NaN,NaN
15555106,93316,2000-03-14,1800.0,15.0625,15.0625,15.375,15.3750,NaN,NaN
15555107,93316,2000-03-15,2100.0,15.0625,15.0625,15.375,15.0625,NaN,NaN


In [29]:
filtered_data_10001 = crsp_data_1993_2000_R5[crsp_data_1993_2000_R5["PERMNO"] == 10001]
filtered_data_10001.tail(20)

,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,DlyClose_5_days_later,Log_Return
1557,10001,2000-11-24,400.0,9.1250,9.12500,9.1250,9.1250,9.000,-0.013793
1558,10001,2000-11-27,802.0,9.3750,9.06250,9.3750,9.0625,9.500,0.013245
1559,10001,2000-11-29,700.0,8.8750,8.87500,8.8750,8.8750,9.500,0.068053
1560,10001,2000-11-30,100.0,9.5625,9.56250,9.5625,9.5625,9.750,0.019418
1561,10001,2000-12-01,1000.0,9.1250,9.12500,9.5000,9.5000,9.750,0.066249
1562,10001,2000-12-05,1000.0,9.0000,9.00000,9.0000,9.0000,9.750,0.080043
1563,10001,2000-12-07,2160.0,9.5000,9.00000,9.5000,9.5000,9.750,0.025975
1564,10001,2000-12-08,3865.0,9.5000,9.50000,9.5625,9.5000,9.750,0.025975
1565,10001,2000-12-13,1700.0,9.7500,9.75000,9.7500,9.7500,9.500,-0.025975
1566,10001,2000-12-14,200.0,9.7500,9.75000,9.7500,9.7500,9.750,0.000000


In [30]:
# Drop the remaining NaN in the newly-created crsp_data_1993_2000_R5
crsp_data_1993_2000_R5 = crsp_data_1993_2000_R5.dropna()

# Check the shape again
print(crsp_data_1993_2000_R5.shape) # (15483676, 9)

(15483676, 9)


In [31]:
# We take the real return, since log return is log(r + 1) = log(P_t / P_(t - 1))
# Because of that, we take the exponential, then minus 1
crsp_data_1993_2000_R5["Ret_5d"] = np.expm1(crsp_data_1993_2000_R5["Log_Return"])

crsp_data_1993_2000_R5

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_90371/3784330760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_data_1993_2000_R5["Ret_5d"] = np.expm1(crsp_data_1993_2000_R5["Log_Return"])


,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,DlyClose_5_days_later,Log_Return,Ret_5d
0,10001,1993-01-04,150.0,14.5000,14.5000,14.5000,14.5000,13.7500,-0.053110,-0.051724
1,10001,1993-01-07,228.0,14.5000,14.5000,14.5000,14.5000,13.7500,-0.053110,-0.051724
2,10001,1993-01-08,1375.0,14.5000,14.2500,14.5000,14.2500,14.2500,-0.017392,-0.017241
3,10001,1993-01-11,200.0,14.5000,14.5000,14.5000,14.5000,14.2500,-0.017392,-0.017241
4,10001,1993-01-13,700.0,13.7500,13.7500,14.0000,14.0000,13.7500,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
15555099,93316,2000-03-03,2350.0,15.1250,15.1250,15.3125,15.1250,15.2500,0.008230,0.008264
15555100,93316,2000-03-06,1390.0,15.3125,15.3125,15.3125,15.3125,15.3750,0.004073,0.004082
15555101,93316,2000-03-07,1280.0,15.1250,15.1250,15.3125,15.3125,15.0625,-0.004141,-0.004132
15555102,93316,2000-03-08,400.0,15.3125,15.3125,15.3125,15.3125,15.0625,-0.016461,-0.016327
